## start every session by running this block of code first:

In [1]:
!pip -q install git+https://github.com/mwshinn/PyDDM
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas
import numpy as np

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## then run this code to generate the interactive simulation

definitions of adjustable parameters in this model:
- **coherence**: visual evidence strength / reliability
    - see bottom of page 3 on the definitions tab of your notebook for more info
- **cue**: memory evidence strength / reliability. this value corresponds to how predictive a colored border is for a particular scene image.
- **thinning**: controls the relative "speed" or frequency with which samples are drawn from memory. in the experiment, we control the speed/frequency with which people sample visual evidence -- this is determined by how rapidly the flicker stream alternates between different evidence frames. but we don't have control over how quickly people do an analagous sampling process from their memory for the cue. probably, this value is different for different people and also varies as a function of cue strength. when we go to fit the model, we will do so with different fixed values of memory thinning to examine how robust the parameter estimates are to fluctuations in this relative sampling rate.
- **noise1_duration**: how long the first noise period is (in seconds)
- **signal1_duration**: how long the first signal period is (in seconds)
- **noise2_duration**: how long the second noise period is (in seconds)
- **initial_bound**: value of the threshold at the start of the trial
- **collapse_rate**: how quickly the value of the threshold should change over the course of the trial. to best match the type of model we will begin by fitting, you should set this value to 0.

In [ ]:
def coherence_changes(t, coherence, cue, thinning, noise1_duration, signal1_duration, noise2_duration):
    if t < noise1_duration:
      return cue/thinning
    elif t > noise1_duration and t < noise1_duration + signal1_duration:
      return coherence + (cue/thinning)
    elif t > noise1_duration + signal1_duration and t < noise1_duration + signal1_duration + noise2_duration:
      return cue/thinning
    else:
      return coherence + (cue/thinning)

def boundary(threshold):
  return threshold

m = pyddm.gddm(drift=coherence_changes,
               mixture_coef=0.2,
               T_dur=3.5,
               bound=lambda t,initial_bound,collapse_rate : initial_bound * np.exp(collapse_rate*t),
               nondecision=0,
               parameters={"noise1_duration": (0, 1.25), "signal1_duration": (0, 1), "noise2_duration": (0, 1.25),
                           "coherence": (0.5,1), "cue": (0,1), "thinning": (1,36), "threshold": (0.5,3), "initial_bound": (0.5,5), "collapse_rate":(-5,5)})

pyddm.plot.model_gui_jupyter(m)

#m.show()

Output()

## parameter recovery attempt

After specifying the model, pyddm makes it very easy to simulate data using that model. That's what I do in the first couple lines on this code chunk.

Then, after simulating data, I can perform a model/parameter recovery analysis. This analysis allows me to see how well my model is able estimate the true parameter values of a particular dataset. Here are the steps that allow me to do that:

1. Generate "synthetic" data via model simulations (lines 1-21)
2. Fit the model to the synthetic data (lines 27-30)
3. Compare the fitted/estimated parameter values to the ground truth of those values as defined in the simulation model (lines 33-36)

In [ ]:
# specify generating model
true_model = pyddm.gddm(drift=coherence_changes,
                        mixture_coef=0,
                        T_dur=3.5,
                        bound=1.5,
                        nondecision=0,
                        parameters={"noise1_duration": 0.75, "signal1_duration": 0.75, "noise2_duration": 1,
                           "coherence": 0.7, "thinning": 4},
                        conditions=["cue"])

# simulate 1600 trials with 80% cue
strongCue_sample = true_model.solve(conditions={"cue": 0.8}).sample(1600)

# simulate 1000 trials with 50% cue
neutralCue_sample = true_model.solve(conditions={"cue": 0.5}).sample(1000)

# simulate 400 trials with 20% cue
weakCue_sample = true_model.solve(conditions={"cue": 0.2}).sample(400)

# concatenate into one sample
sim_data = strongCue_sample + neutralCue_sample + weakCue_sample

# visualize model fit over simulated data
pyddm.plot.model_gui_jupyter(true_model, sim_data)

# specify fitting model
fitting_model = pyddm.gddm(drift='cue', mixture_coef=0, T_dur=4, bound=1.5, nondecision=0, starting_position=0, dx=0.001, dt=0.001, parameters={'cue': (0.1, 0.9)})

# apply the fitting model to simulated data
fitting_model.fit(data, lossfunction=pyddm.LossBIC, verbose=False)

# get printout of results
fitting_model.show()

# get fixed & fitted parameter values
fitting_model.parameters()

# visualize fitting success
pyddm.plot.plot_fit_diagnostics(model=fitting_model, sample=data)

Output()

## simulation round 2: model_4drifts

In [ ]:
def coherence_changes(t, trueCongruence, noise1frames_obs, noise2frames_obs, signal1frames_obs, signal2frames_obs, noise1_drift, signal1_drift, noise2_drift, signal2_drift):
    if t < noise1frames_obs:
      if trueCongruence == 'congruent':
        return noise1_drift
      else:
        return -noise1_drift
    elif t > noise1frames_obs and t < noise1frames_obs + signal1frames_obs:
      if trueCongruence == 'congruent':
        return signal1_drift
      else:
        return -signal1_drift
    elif t > noise1frames_obs + signal1frames_obs and t < noise1frames_obs + signal1frames_obs + noise2frames_obs:
      if trueCongruence == 'congruent':
        return noise2_drift
      else:
        return -noise2_drift
    else:
      if trueCongruence == 'congruent':
        return signal2_drift
      else:
        return -signal2_drift

model_4drifts = pyddm.gddm(
    drift = coherence_changes,
    name = "4drifts",
    starting_position = 0,
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'B': (0.01, 4), 'ndt': (0, 0.5),
                'noise1_drift': (0, 1), 'signal1_drift': (0,1), 'noise2_drift': (0,1), 'signal2_drift': (0,1)},
    conditions = ['trueCongruence', 'noise1frames_obs', 'noise2frames_obs', 'signal1frames_obs', 'signal2frames_obs']
)

# vizualize the model
pyddm.plot.model_gui_jupyter(model_4drifts, conditions={"trueCongruence": ['congruent', 'incongruent'],
                                                        'noise1frames_obs': [0.4, 0.6],
                                                        'signal1frames_obs': [0.3, 0.7],
                                                        'noise2frames_obs': [0.4, 0.6],
                                                        'signal2frames_obs': [0.2, 0.4]})

Output()

## simulation round 3: multisensory integration example

In [ ]:
def dynamic_drift(t, trueCongruence, memoryEvidence, memoryWeight, visionEvidence, visionWeight, noise1frames_obs, noise2frames_obs, signal1frames_obs, signal2frames_obs):
    if t < noise1frames_obs:
      if trueCongruence == 'congruent' or trueCongruence == 'neutral':
        return memoryEvidence*memoryWeight
      else:
        return -memoryEvidence*memoryWeight
    elif t > noise1frames_obs and t < noise1frames_obs + signal1frames_obs:
      if trueCongruence == 'congruent' or trueCongruence == 'neutral':
        return memoryEvidence*memoryWeight + visionEvidence*visionWeight
      else:
        return visionEvidence*visionWeight - memoryEvidence*memoryWeight
    elif t > noise1frames_obs + signal1frames_obs and t < noise1frames_obs + signal1frames_obs + noise2frames_obs:
      if trueCongruence == 'congruent' or trueCongruence == 'neutral':
        return memoryEvidence*memoryWeight
      else:
        return -memoryEvidence*memoryWeight
    else:
      if trueCongruence == 'congruent' or trueCongruence == 'neutral':
        return memoryEvidence*memoryWeight + visionEvidence*visionWeight
      else:
        return visionEvidence*visionWeight - memoryEvidence*memoryWeight


model = pyddm.gddm(
    drift = dynamic_drift,
    starting_position = 0,
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'B': (0.01, 4), 'ndt': (0, 0.5),
                'memoryWeight': (0,1), 'visionWeight': (0,1)},
    conditions = ['trueCongruence', 'memoryEvidence', 'visionEvidence', 'noise1frames_obs', 'noise2frames_obs', 'signal1frames_obs', 'signal2frames_obs']
)

# vizualize the model
pyddm.plot.model_gui_jupyter(model, conditions={"trueCongruence": ['congruent', 'incongruent'],
                                                        'memoryEvidence': [0, 0.5, 0.8],
                                                        'visionEvidence': [0, 0.5, 0.7],
                                                        'noise1frames_obs': [0.4, 0.6],
                                                        'signal1frames_obs': [0.3, 0.7],
                                                        'noise2frames_obs': [0.4, 0.6],
                                                        'signal2frames_obs': [0.2, 0.4]})


Output()